### Imports

In [2]:
# modules utilisés
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import xgboost
import sklearn
import warnings

from time import time
from dython.nominal import associations

# imports SKLEARN
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, validation_curve, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score, mean_squared_log_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, KFold, cross_validate, RandomizedSearchCV, cross_val_predict
from sklearn.feature_selection import RFECV

# import de la fonction TargetEncoder()
from category_encoders.target_encoder import TargetEncoder

### Divers

In [10]:
# liste des colonnes de nos df de présentations des résultats
LISTE_COL = ["Model", "Fit_Time", "Score_time", "MAE", "RMSLE", "R2"] 

# Métriques utilisées en temps normal et au cas ou la MSLE ne soit pas adaptée
SCORING = ["neg_mean_absolute_error", "neg_mean_squared_log_error", "r2"]
SCORING_alt = ["neg_mean_absolute_error", "r2"]

# facteur aléatoire que nous utiliserons le long de notre travail
SEED = 47

# nombre de "folds" lors des validations croisées
K = 5

# instruction évitant les avertissements, nombreux lors de l'utilisation du modèle de TargetEncoder
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# augmentation du nombre de colonnes d'un objet DataFrame à visualiser
pd.options.display.max_columns = 30

# paramètres matplotlib & seaborn
font = {'family' : 'verdana',
        'weight' : 'normal',
        'size'   : 10}
plt.rc('font', **font)

sns.set(font_scale=0.9)

# fonction de modélisation
def modelize(model, name, X, y, return_model = False , graph = False, k = K) :
        
    resultats = []
    
    kf = KFold(k, shuffle = True, random_state = SEED)
    
    try :
        cv_res = cross_validate(model, X, y, scoring = SCORING, cv = kf)

        resultats.append(name)
        resultats.append(cv_res["fit_time"].sum())
        resultats.append(cv_res["score_time"].sum())
        resultats.append(-cv_res['test_neg_mean_absolute_error'].mean())
        resultats.append(np.sqrt(-cv_res['test_neg_mean_squared_log_error']).mean())
        resultats.append(cv_res['test_r2'].mean())

    
    # Si, comme cela arrive parfois, la RMSLE est impossible à calculer, on lui donne la valeur 0.
    except :
        
        cv_res = cross_validate(model, X, y, scoring = SCORING_alt, cv = kf)
        
        resultats.append(name)
        resultats.append(cv_res["fit_time"].sum())
        resultats.append(cv_res["score_time"].sum())
        resultats.append(-cv_res['test_neg_mean_absolute_error'].mean())
        resultats.append(0)  
        resultats.append(cv_res['test_r2'].mean())
    
    # affichage de graphs au choix
    if graph :

        pred = cross_val_predict(model, X, y, cv = kf)

        error = np.abs(y - pred)

        fig = plt.figure(figsize = (8, 8))
        plt.subplot(2,2,1)
        plt.scatter(y, pred, color = "coral")
        plt.subplot(2,2,2)
        plt.hist(error, bins = 50)
        plt.show()
            
    new_row = {k:v for k, v in zip(LISTE_COL, resultats)}

    # récupération d'un modèle entrainé au choix
    if return_model :
        
        return new_row, model.fit(X, y)
        
    else :        
    
        return new_row
    
    
# chargement de nos données avec et sans outliers
X = pickle.load(open("Data/X.pickle", "rb"))
y = pickle.load(open("Data/y.pickle", "rb"))

X_no = pickle.load(open("Data/X_no.pickle", "rb"))
y_no = pickle.load(open("Data/y_no.pickle", "rb"))

# Cible du second modèle (variable "GHGEmissions(MetricTonsCO2e)")
y = y["GHGEmissions(MetricTonsCO2e)"]
y_no = y_no["GHGEmissions(MetricTonsCO2e)"]

# transformation log(p+1) des cibles
y = y.transform(np.log1p)
y_no = y_no.transform(np.log1p)

### Récup données

In [44]:
data = pickle.load(open("Data/red_level_tar_4.pickle", "rb"))

"data" est une liste avec plein de trucs issus de mon "script réducteur de variables catégorielles".<br><br>Les var catégorielles ont été réduites si cela améliorait le modèle. Voici un genre de "log" de l'opération.

In [45]:
data[0][0]

[(55, 12, 0.8134406356697665, 44),
 (51, 0, 0.8134406356697665, 51),
 (43, 7, 0.814127810625789, 37),
 (24, 22, 0.8151861968733349, 3),
 (13, 10, 0.8154173233351696, 4),
 (12, 3, 0.8166644792719373, 10),
 (7, 0, 0.8166644792719373, 7),
 (2, 0, 0.8166644792719373, 2),
 (2, 0, 0.8166644792719373, 2),
 (2, 1, 0.8167345858786202, 1),
 (2, 1, 0.8167740740381724, 1)]

Une variable qui avait 55 catégories à été réduite à 44, une qui en avait de 51 n'a pas bougé, la 3ème qui en avait 43 est passée à 37, etc. Et à chaque fois on voit de combien le R2 est amélioré par ces changements.

Puis à la fin je récupère le df optimisé qui est celui-ci.

In [46]:
df = data[0][4][-1].copy()

In [47]:
df.head(3)

,NumberOfPropertyUse,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyBuildingGFARate,L_PUTGFA,S_PUTGFA,T_PUTGFA,ENERGYSTARScore,Neighborhood,BuildingType,PrimaryPropertyType,LargestPropertyUseType,SecondLargestPropertyUseType,ThirdLargestPropertyUseType,DecadeBuilt,Steam,NaturalGas,Other,Outlier
0,0.405427,-0.149047,0.125401,-0.157012,0.350543,-0.282684,0.746611,-0.334138,0.304832,DOWNTOWN,Multifamily LR (1-4),Low-Rise Multifamily,Multifamily Housing,Retail Store,No Data,1900,1,1,0,0
1,-0.851996,-0.149047,0.125401,-0.069383,0.350543,0.555618,-0.737749,-0.334138,0.497360,No_Data,Multifamily LR (1-4),Low-Rise Multifamily,Multifamily Housing,No Data,No Data,2000,0,0,0,0
2,-0.851996,-0.149047,0.125401,0.818316,0.350543,-4.655013,-0.737749,-0.334138,0.602399,DOWNTOWN,NonResidential,No_Data,Retail Store,No Data,No Data,1980,0,0,0,0


Comme vu ds les deux dernières lignes du log, les deux variables binaires tout au bout, **Other** et **Outlier**, ont été réduites à **une seule valeur**, ce sont des constantes avec plus que des **0**.

In [48]:
df.describe(include = "category")

,Neighborhood,BuildingType,PrimaryPropertyType,LargestPropertyUseType,SecondLargestPropertyUseType,ThirdLargestPropertyUseType,DecadeBuilt,Steam,NaturalGas,Other,Outlier
count,3303,3303,3303,3303,3303,3303,3303,3303,3303,3303,3303
unique,4,7,3,44,51,37,10,2,2,1,1
top,No_Data,NonResidential,No_Data,Multifamily Housing,No Data,No Data,2000,0,1,0,0
freq,1868,1449,1773,1657,1663,2721,495,3174,2059,3303,3303


Modélisons à partir de ce DF...

D'abord l'encodage...

In [49]:
cat_liste = df.columns[df.dtypes == "category"]

for cat in cat_liste :
    tar_enc = TargetEncoder()
    df[cat] = tar_enc.fit_transform(df[cat], y)

Les var catégorielles ont bien été encodées y compris les deux du fonds qui ont bien une valeur cst.

In [50]:
df[["Other", "Outlier"]].describe()

,Other,Outlier
count,3303.000000,3303.000000
mean,3.550852,3.550852
std,0.000000,0.000000
min,3.550852,3.550852
25%,3.550852,3.550852
50%,3.550852,3.550852
75%,3.550852,3.550852
max,3.550852,3.550852


Je modélise...

In [51]:
res, mod = modelize(GradientBoostingRegressor(random_state = SEED), "Gradient Boosting", df, y, True)
res

{'Model': 'Gradient Boosting',
 'Fit_Time': 1.5310020446777344,
 'Score_time': 0.009999990463256836,
 'MAE': 0.43974940296133347,
 'RMSLE': 0.1419046234788987,
 'R2': 0.8167740740381724}

Ceci est le résultat attendu, le même qu'obtenu à la fin par mon script (dernière ligne du log).

On remarque que dans ce calcul, le modèle accords bien 0 importance aux deux dernières variables constantes.

In [52]:
mod.feature_importances_

array([0.00050885, 0.00075628, 0.00815359, 0.3531091 , 0.00247797,
       0.01028575, 0.00287398, 0.00172117, 0.02804715, 0.00371393,
       0.0038875 , 0.00094226, 0.10822685, 0.00342635, 0.0020597 ,
       0.00387174, 0.03682785, 0.42910997, 0.        , 0.        ])

Du coup, ces deux variables inutiles, je veux les virer...

Faisons-le par exemple depuis le df déjà "target-encodé"...

In [53]:
df = df.drop(["Other", "Outlier"], axis = 1)

et si maintenant je re-modélise...

In [54]:
res, mod = modelize(GradientBoostingRegressor(random_state = SEED), "Gradient Boosting", df, y, True)
res

{'Model': 'Gradient Boosting',
 'Fit_Time': 1.4800021648406982,
 'Score_time': 0.029999971389770508,
 'MAE': 0.44012398922400625,
 'RMSLE': 0.1419883657327466,
 'R2': 0.8163584622842832}

Résultat différent.........................

Ce truc me rend juste fou depuis 3 jours...

Est-ce que comme ça, à vu de nez, tu vois où est le problème ?

Merci^^ !